<a href="https://colab.research.google.com/github/Alih111/naivee-and-logistic-nlp/blob/main/NLP_naivebayes_logisticRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets
from datasets import load_dataset
sst_dataset=load_dataset("sst")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.2 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1461: FutureWarning: The repository for sst contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/sst
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split:   0%|          | 0/8544 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1101 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2210 [00:00<?, ? examples/s]

getting training data sentences and labels


In [2]:
train_data = sst_dataset["train"]
sentences=train_data["sentence"]
sentences=[s.lower() for s in sentences]

classify sentences according to score

•From 0 to 0.2 (0.2 included) will be class 0 “very negative”.
•From 0.2 to 0.4 (0.4 included) will be class 1 “negative”.
•From 0.4 to 0.6 (0.6included)will be class 2 “neutral”.•From 0.6to 0.8 (0.8 included)will be class 3 “positive”.•From 0.8to 1.0(1.0included)will be class 4 “very positive”.

In [3]:
import numpy as np
scores= train_data["label"]
classes = [[] for _ in range(5)]
#put each sentence in the class that it belongs to
i=0
for score in scores:
    sentences[i]=sentences[i][:-1]
    if score <= 0.2:
      scores[i]=0
      classes[0].append(sentences[i])
    elif score > 0.2 and score <= 0.4:
      scores[i]=1
      classes[1].append(sentences[i])
    elif score > 0.4 and score <=0.6:
      scores[i]=2
      classes[2].append(sentences[i])
    elif score > 0.6 and score <=0.8:
      scores[i]=3
      classes[3].append(sentences[i])
    elif score > 0.8 and score <=1:
      scores[i]=4
      classes[4].append(sentences[i])
    i+=1
print(sentences[0])

the rock is destined to be the 21st century 's new `` conan '' and that he 's going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal 


count number of occurences of all words in the document


In [4]:
from collections import Counter

sentences_np=np.array(sentences)
all_words = ' '.join(sentences_np).split()
word_counts = Counter(all_words)
word_counts_dict = dict(word_counts)
del word_counts
del sentences_np
del all_words
print(len(word_counts_dict))

16677


naivee bayes

In [5]:
def countWord(goal,words):#count words in a single class
    return  np.sum(words == goal)

training loop


In [6]:
from math import log
words_probablites={0: {},#will store the likelihood of each word in each class
                  1: {},
                   2: {},
                   3: {},4: {}}
Ndoc = sum(len(row) for row in classes)# counts the total number of words
print(Ndoc)
stopWords=['the' ,'is','are','you','he','she','it','they','a','am','an','that','to','and','or','be','can','of','was','with','on','his','her','from','at','as','in','can']
priors=[]
for k in range(len(classes)):
  Nc=len(classes[k])#count total number of words in each class
  priors.append(log(Nc/Ndoc))
  myclass = ' '.join(classes[k]).split()#get vocubulary for each class
  word_counts = Counter(myclass)
  this_word_counts_dict = dict(word_counts)#stores words count in each class to measure likelihood
  for word in this_word_counts_dict:
      if word not in words_probablites[k]  and word not in stopWords:#and len(word) > 1
        #occurence=countWord(word,words)
        log_likelihood=(log((this_word_counts_dict[word]+1)/(word_counts_dict[word]+1)))
        words_probablites[k][word]=log_likelihood

  del this_word_counts_dict



8544


test naivee bayes


In [7]:
final_sum= [[] for _ in range(5)] #will contian the probability of each sentence to be in each class
j=0
print(len(words_probablites[3]))
for i in range (len(classes)):
  sum1=priors[i]
  for sentence in sentences:
      words=sentence.split(' ')
      m=0
      for word in words:#measures the probabily of each sentence for a specific class
        #if len(word) > 1:#skip one letter words
          if word in words_probablites[i] and (m ==0 or not word[0].isupper()):#check if word appeared before in this class
            sum1+=words_probablites[i][word]
          m+=1
      final_sum[i].append(sum1)

  j+=1
final_sum=np.array(final_sum)
print(final_sum[:,0])
max_indices = np.argmax(final_sum, axis=0)



7986
[-30.83016152 -19.34766005 -29.34274745 -22.01492355 -27.02556589]




```
# This is formatted as code
```

sklearn naivee bayes



In [8]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
model=make_pipeline(CountVectorizer(min_df=0),MultinomialNB(alpha=1))
model.fit(sentences,scores)
labels = model.predict(sentences)
labels[0]

3

confusion matrix

In [19]:

class ConfusionMatrix:
  def __init__(self,actual,predicted):
    self.confMat=np.zeros((5,5))
    self.true=0
    self.false=0
    self.t0=0
    for k in range(len(actual)):#predicted row and actual column
      i=predicted[k]#predicted label
      j=actual[k]#actual label
      self.confMat[i][j]+=1
      if i == j:
        self.true+=1
      else:
        self.false+=1
  def accuracy(self):
    return self.true/(self.true+self.false)


  def precision(self):
    precisions = []
    for i in range(len(self.confMat)):
        true_positives = self.confMat[i][i]
        false_positives = sum(self.confMat[j][i] for j in range(len(self.confMat)) if j != i)
        if true_positives + false_positives == 0:
            precisions.append(0)
        else:
            precisions.append(true_positives / (true_positives + false_positives))
    return precisions
  def recall(self):
    recall = []
    for i in range(len(self.confMat)):
        true_positives = self.confMat[i][i]
        false_negative = sum(self.confMat[i,j] for j in range(len(self.confMat)) if j != i)
        if true_positives + false_negative == 0:
            recall.append(0)
        else:
            recall.append(true_positives / (true_positives + false_negative))
    return recall
  def F1Score(self):
    precision = np.array(self.precision())
    recall = np.array(self.recall())
    f1_score = 2 * (precision * recall) / (precision + recall)
    return f1_score

sklearnMethod=ConfusionMatrix(actual=scores,predicted=labels)
myMethod=ConfusionMatrix(actual=scores,predicted=max_indices)

print(f"my method accuracy = {myMethod.F1Score()} sklearn accuracy = {sklearnMethod.F1Score()}")
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
f1=f1_score(labels,scores,average=None)
recall = recall_score(max_indices,scores,average=None)
print(recall)
print(f1)

my method accuracy = [       nan        nan        nan 0.42726854        nan] sklearn accuracy = [0.72335601 0.80256261 0.76789588 0.82712255 0.7760109 ]
[0.         0.         0.         0.27176843 0.        ]
[0.72335601 0.80256261 0.76789588 0.82712255 0.7760109 ]


<ipython-input-19-d2ef4463f0f9>:42: RuntimeWarning: invalid value encountered in divide
  f1_score = 2 * (precision * recall) / (precision + recall)
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Logistic Regression

Feature Representation

In [10]:
from ctypes import BigEndianStructure
def generate_word_bi_grams(sentences):
    # Create a set to store unique word bi-grams
    unique_bi_grams = set()

    # Iterate through each sentence to extract word bi-grams
    for sentence in sentences:
        words = sentence.split()
        words= [word for word in words if word.isalpha()]
        bi_grams = zip(words, words[1:])
        unique_bi_grams.update(bi_grams)

    # Convert the set of unique word bi-grams to a sorted list

    return unique_bi_grams

In [11]:
bi_grams = generate_word_bi_grams(sentences)
len(bi_grams)

75149

In [12]:
def generate_sparse_feature_representation(sentences, unique_bi_grams):
    # Initialize a list to store the feature representation for each sentence
    feature_representation = []
    # Iterate through each sentence to create the sparse feature vector
    print('a')
    for sentence in sentences:
        words = sentence.split()
        bi_grams = set(zip(words, words[1:]))

        # Create a binary vector for the current sentence
        sentence_vector = [1 if bi_gram in bi_grams else 0 for bi_gram in unique_bi_grams]
        feature_representation.append(sentence_vector)

    return feature_representation

In [13]:
features = generate_sparse_feature_representation(sentences, bi_grams)

a


KeyboardInterrupt: 

In [ ]:
# each sentence has a vector of features of length
len(features[0])

sigmoid Function


In [ ]:

def sigmoid(x):
  return (1/(1+np.exp(-x)))
# turn scores to softmax matrix
def turnScores(scores):
  new_scores=np.zeros((len(scores),5))
  i=0
  for score in scores:
    new_scores[i,score]=1
    i+=1
  return new_scores
def predict(weights,bias,x):
  z=np.dot(weights,x)

In [ ]:
# z = WX+B
b=np.zeros((5))#bias for each sentence
new_scores=turnScores(scores)
print(new_scores[0])
W=np.zeros((5,len(features[0])))
W.shape

In [ ]:
def multiClassLogistic(scores,features,learningRate,num_epochs,sample_size,num_of_samples):
  b=np.zeros((5))#bias for each sentence
  W=np.zeros((5,len(features[0])))
  for epoch in range(num_epochs):
      random_samples = np.random.uniform(0, len(features), 64)
      for sample in random_samples:
        X=features[:,sample]
        Y=scores[:,sample]

